In [493]:
from bs4 import BeautifulSoup
import requests

In [442]:
url = 'https://www.mobcraftbeer.com/recipes/previous_winners'     # Web site to be scraped
html = requests.get(url).text                                     # Getting the html text
soup = Soup(html)

In [443]:
soup = BeautifulSoup(html, 'lxml')
mylink = soup.find_all('div',{'class':'winner-block'})

In [444]:
# Collecting all href links to be used later in creating our Recipebook
Beerlist = []                                                             # Create an empty list to be appended to later
for x in mylink:                                                          # Loop through the links
    Beer = []
    for a in x.find_all('a', href=True):                                  # Finding all 'a' tags that has a href link
        beerlink = a['href']                                              # Saving link to a var
        if beerlink[:8] == '/recipes':                                    # Using an If statement to filter out unwanted links
            Beer.append(beerlink)                                         # Append link to Beer
    Beer = str(set(Beer)).replace('{','').replace('}','').replace("'","") # Saving only one copy of the link, and remove brackets and '
    Beerlist.append(Beer)

In [445]:
base = 'https://www.mobcraftbeer.com'
print(base + Beerlist[4])

https://www.mobcraftbeer.com/recipes/the-kneads-of-the-bread-outweigh-the-needs-of-the-few


In [446]:
Beerlist = [base + str(x) for x in Beerlist]     # List comprehension that adds the base url and the url we just got from Beerlist together to use in scraping recipes

['https://www.mobcraftbeer.com/recipes/bnfs-butter-nutter-finger-stout',
 'https://www.mobcraftbeer.com/recipes/eissturm',
 'https://www.mobcraftbeer.com/recipes/atoqua',
 'https://www.mobcraftbeer.com/recipes/pinky-up',
 'https://www.mobcraftbeer.com/recipes/the-kneads-of-the-bread-outweigh-the-needs-of-the-few',
 'https://www.mobcraftbeer.com/recipes/sour-catch',
 'https://www.mobcraftbeer.com/recipes/thaid-up',
 'https://www.mobcraftbeer.com/recipes/field-of-creme-stout',
 'https://www.mobcraftbeer.com/recipes/miltown-milkshake',
 'https://www.mobcraftbeer.com/recipes/wishilicious-blueberry-shandy',
 'https://www.mobcraftbeer.com/recipes/raspberry-pillows',
 'https://www.mobcraftbeer.com/recipes/jamacican-me-coconutz',
 'https://www.mobcraftbeer.com/recipes/caramel-brownie-sundae-porter',
 'https://www.mobcraftbeer.com/recipes/chocolate-covered-bourbon-cherry-stout',
 'https://www.mobcraftbeer.com/recipes/treestand-juicebox-wipa',
 'https://www.mobcraftbeer.com/recipes/mahalo-mallow

In [447]:
#Start scraping a test page to see what elements will need to be grabbed.
url = 'https://www.mobcraftbeer.com/recipes/the-kneads-of-the-bread-outweigh-the-needs-of-the-few'
html = requests.get(url).text
soup = Soup(html)
soup = BeautifulSoup(html, 'lxml')
description = soup.find_all('div',{'class':'single-recipe-content'})
recipe = soup.find('div',{'class':'recipe-attributes-wrapper'})

In [494]:
def transform(multilevelDict):
    '''enter a multiplebelDiction and it will turn the htlm tags from key and values'''
    return {str(key).replace("<h6>","").replace("</h6>","") : (transform(value) if isinstance(value, dict) else value) for key, value in multilevelDict.items()}

In [492]:
def GetBeerRecipes(inputURL):
    url = inputURL                                                                       # Setting the URL var for BS4
    html = requests.get(url).text                                                        # Requesting the text from the URL
    soup = BeautifulSoup(html, 'lxml')                                                   # Parsing the HTLM
    description = soup.find_all('div',{'class':'single-recipe-content'})                 # Getting the Description of the Beer                 
    results = {}                                                                         # Creating an empty Dictionary for all of the Recipes
    results['Name'] = name = inputURL.split('/')[-1].replace('-',' ')                    # Getting Name from the last part of the URL 
    for i in description:                                                                # Looping through all of Page HTLM block
        recipe = soup.find('div',{'class':'recipe-attributes-wrapper'})                  # Getting the block that contains the details for the recipe
        listing = i.get_text().split("Recipe Details",1)[0]                              # The Beer decription came in a long string of HTLM. used String slicing take everthing before 'Recipe Details'
        results['Description'] = listing                                                 # Adding the Beer descrip to the Results Dictionary
        for x in recipe:                                                                 # Looping through the recipe block of htlm to get the ingredients to make the beer 
            results_li = []                                                              # Create an empty list for all of the List Items
            li = x.find_all()                                                            # Find all of the List Items
            for y in li:                                                                 # Loop through all of the List Items
                results_li.append(str(y.next).replace("<li>","").replace("</li>",""))    # Append and clean the format for all of the List Itmes
            results[x.next] = set(results_li)                                            # Add X.next as the key value and a set (to remove dups) of cleaned List Item 
    return transform(results)                                                            # Send the final list through transform to clean the text formatting                                                      

In [489]:
RecipeBook = [GetBeerRecipes(x) for x in Beerlist]

In [491]:
RecipeBook

[{'Name': 'bnfs butter nutter finger stout',
  'Description': 'A chocolate peanut butter stout with cacao nibs, butterfingers, nutter butters and peanut butter.',
  'Special ingredients': {'1oz Defatted Peanut Powder @ Secondary',
   '4oz butterfingers @ Secondary',
   '4oz cocoa nibs @ Secondary',
   '4oz nutter butters @ Secondary',
   '8oz Dextrose Sugar @ Boil',
   'Special ingredients'},
  'Yeast': {'Wyeast Scottish Ale', 'Yeast'},
  'Malt bill': {'20% Maris Otter Pale Malt',
   '20% Vienna Malt',
   '40% Pale Malt',
   '5% Carapils Malt',
   '5% Chocolate Malt',
   '5% Midnight Wheat',
   '5% Victory',
   'Malt bill'},
  'Boil time': {'60', 'Boil time'},
  'Hop regimen': {'Hop regimen', 'Nugget @ 60 (21 IBUs)'},
  'Gravity': {'23P/6P', 'Gravity'},
  'IBU': {'21', 'IBU'},
  'ABV': {'9.5%', 'ABV'}},
 {'Name': 'eissturm',
  'Description': 'When translated from German means “ice storm.”  German eisbock aged in oak barrels.',
  'Special ingredients': {'Oak Barrels', 'Special ingredien

## Working on changing the List comprehension into a Dictionary comprehension

In [ ]:
Booktest = {range(len(Beerlist))}

In [ ]:
print(type(RecipeBook))
# clean = {RecipeBook[key].remove(key) for key in RecipeBook if key in RecipeBook[key]}
# clean =   {vals.remove(val) for key,vals in RecipeBook.items() for val in vals if val == key}
# if key in RecipeBook[key]:
#     RecipeBook[key].remove(key)


In [490]:
RecipeBook

[{'Name': 'bnfs butter nutter finger stout',
  'Description': 'A chocolate peanut butter stout with cacao nibs, butterfingers, nutter butters and peanut butter.',
  'Special ingredients': {'1oz Defatted Peanut Powder @ Secondary',
   '4oz butterfingers @ Secondary',
   '4oz cocoa nibs @ Secondary',
   '4oz nutter butters @ Secondary',
   '8oz Dextrose Sugar @ Boil',
   'Special ingredients'},
  'Yeast': {'Wyeast Scottish Ale', 'Yeast'},
  'Malt bill': {'20% Maris Otter Pale Malt',
   '20% Vienna Malt',
   '40% Pale Malt',
   '5% Carapils Malt',
   '5% Chocolate Malt',
   '5% Midnight Wheat',
   '5% Victory',
   'Malt bill'},
  'Boil time': {'60', 'Boil time'},
  'Hop regimen': {'Hop regimen', 'Nugget @ 60 (21 IBUs)'},
  'Gravity': {'23P/6P', 'Gravity'},
  'IBU': {'21', 'IBU'},
  'ABV': {'9.5%', 'ABV'}},
 {'Name': 'eissturm',
  'Description': 'When translated from German means “ice storm.”  German eisbock aged in oak barrels.',
  'Special ingredients': {'Oak Barrels', 'Special ingredien